In [85]:
!conda env list
!conda activate Python36
!conda env list

# conda environments:
#
base                     C:\ProgramData\Anaconda3
Coursera_DL_202109       C:\Users\koba0\.conda\envs\Coursera_DL_202109
Python36                 C:\Users\koba0\.conda\envs\Python36
ml_framework          *  C:\Users\koba0\.conda\envs\ml_framework
ptn37env                 C:\Users\koba0\.conda\envs\ptn37env

# conda environments:
#
base                     C:\ProgramData\Anaconda3
Coursera_DL_202109       C:\Users\koba0\.conda\envs\Coursera_DL_202109
Python36                 C:\Users\koba0\.conda\envs\Python36
ml_framework          *  C:\Users\koba0\.conda\envs\ml_framework
ptn37env                 C:\Users\koba0\.conda\envs\ptn37env



In [86]:
!python --version


Python 3.10.4


In [87]:
#!python -m pip install --upgrade pip
#!pip install tensorflow

In [138]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold


img_rows, img_cols = 28, 28
num_channels = 1
num_classes = 10


def prep_data(raw):
    y = raw[:,0]
    out_y = tf.keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    #out_x = x.reshape(num_images, img_rows, img_cols, num_channels)
    out_x = x.reshape(num_images, -1)
    out_x = out_x / 255
    return out_x, out_y

df_train = pd.read_csv("../data/fashion-mnist_train.csv")
df_test = pd.read_csv("../data/fashion-mnist_test.csv")
y_train_onecolumn = np.array(df_train)[:,0]
y_test_onecolumn = np.array(df_test)[:,0]
x_train, y_train = prep_data(np.array(df_train))
x_test, y_test = prep_data(np.array(df_test))

%matplotlib inline
from matplotlib import pyplot as plt
for i in range(10):
    plt.imshow(x_train[i], interpolation='nearest')
    plt.show()

In [89]:
3

3

In [90]:
num_images = x_train.shape[0]

In [91]:
#import tensorflow as tf


#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Полносвязная нейросеть

In [92]:
def neural_network(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model

In [93]:
model = neural_network((784,))
#model.load_weights('neural_networks.h5')
model.fit(
    x=x_train,
    y=y_train,
    batch_size=None,
    epochs=20,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=True,
)
answer1 = model.count_params()
_, score = model.evaluate(x_test, y_test, verbose=0)
answer2 = round(score, 3)

In [94]:
answer1, answer2

(203530, 0.786)

Сверточная нейросеть

In [120]:
def conv_network(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(13, 13, 32)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))

    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model

In [121]:
model1 = conv_network((28,28,1))
model1.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 1600)              0         
                                                                 
 dense_58 (Dense)            (None, 64)              

In [122]:
x_train_matr = x_train.reshape(num_images, img_rows, img_cols, num_channels)
num_test_images = x_test.shape[0]
x_test_matr = x_test.reshape(num_test_images, img_rows, img_cols, num_channels)
x_train_matr.shape, x_test_matr.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [123]:
model1 = conv_network((28,28,1))
model1.fit(
    x=x_train_matr,
    y=y_train,
    batch_size=None,
    epochs=20,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=True,
)
answer3 = model1.count_params()
print(answer3)
_, score = model1.evaluate(x_test_matr, y_test, verbose="auto")
answer4 = round(score, 3)
print(answer4)

Epoch 1/20
1875/1875 [==============================] - 43s 23ms/step - loss: 2.2792 - accuracy: 0.1810
Epoch 2/20
1875/1875 [==============================] - 47s 25ms/step - loss: 2.1843 - accuracy: 0.3336
Epoch 3/20
1875/1875 [==============================] - 48s 26ms/step - loss: 2.0552 - accuracy: 0.4285
Epoch 4/20
1875/1875 [==============================] - 47s 25ms/step - loss: 1.8753 - accuracy: 0.4863
Epoch 5/20
1875/1875 [==============================] - 48s 25ms/step - loss: 1.6605 - accuracy: 0.5125
Epoch 6/20
1875/1875 [==============================] - 48s 26ms/step - loss: 1.4573 - accuracy: 0.5569
Epoch 7/20
1875/1875 [==============================] - 49s 26ms/step - loss: 1.2918 - accuracy: 0.5971
Epoch 8/20
1875/1875 [==============================] - 50s 27ms/step - loss: 1.1641 - accuracy: 0.6425
Epoch 9/20
1875/1875 [==============================] - 48s 26ms/step - loss: 1.0686 - accuracy: 0.6739
Epoch 10/20
1875/1875 [==============================] - 49s 26m

PCA

In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
cv = StratifiedKFold(n_splits=4)
scores = []
for n in range(10, 110, 10):
    lr_model = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n, svd_solver="randomized", random_state=123)),
        ('clf', LogisticRegression(random_state=42))
    ])

    scores.append(cross_val_score(lr_model, x_train, y_train_onecolumn, cv=cv, scoring='accuracy').mean())

C:\Users\koba0\.conda\envs\ml_framework\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\koba0\.conda\envs\ml_framework\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [125]:
scores

[0.7503833333333333,
 0.7979333333333334,
 0.81195,
 0.8206333333333333,
 0.8279833333333333,
 0.8334333333333332,
 0.8368499999999999,
 0.8393666666666667,
 0.8421500000000001,
 0.8429333333333334]

CPU times: total: 1min 5s
Wall time: 15.2 s


C:\Users\koba0\.conda\envs\ml_framework\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8294

Логистическая регрессия


In [113]:
%%time
n=50

pca = PCA(n_components=n, svd_solver="randomized", random_state=123)
scaler = StandardScaler()
pca.fit(x_train)
scaler.fit(x_train)
lr_model = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n, svd_solver="randomized", random_state=123)),
        ('clf', LogisticRegression(random_state=42))
    ])
lr_model.fit(x_train, y_train_onecolumn)
lr_pca_score = lr_model.score(x_test, y_test_onecolumn)
lr_pca_score

C:\Users\koba0\.conda\envs\ml_framework\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\koba0\.conda\envs\ml_framework\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

0.8542833333333333

Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [151]:
%%time
n=50

pca = PCA(n_components=n, svd_solver="randomized", random_state=123)
scaler = StandardScaler()
pca.fit(x_train)
scaler.fit(x_train)
gbc_model = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n, svd_solver="randomized", random_state=123)),
        ('clf', GradientBoostingClassifier(random_state=42))
    ])
gbc_model.fit(x_train, y_train_onecolumn)
gbc_pca_score = lr_model.score(x_test, y_test_onecolumn)
gbc_pca_score

CPU times: total: 25min 4s
Wall time: 24min 32s


0.8294

Случайный лес

In [153]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time
scores = dict()
est_arr = [1,5,10,20,30]
n=50
n_est=20
pca = PCA(n_components=n, svd_solver="randomized", random_state=123)
scaler = StandardScaler()
pca.fit(x_train)
scaler.fit(x_train)
for n_est in est_arr:
    rfc_model = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=n, svd_solver="randomized", random_state=123)),
            ('clf', RandomForestClassifier(n_estimators=n_est, random_state=42))
        ])
    rfc_model.fit(x_train, y_train_onecolumn)
    scores[n_est] = lr_model.score(x_test, y_test_onecolumn).copy()
scores